1) Формулировка гипотезы: 
Если изменить дизайн приложения для Android, то количество отказов от поездок на устройствах Android уменьшится на 20%.
Ожидание после тестирования: ниже отмены на Android устройствах.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('new_dataframe.csv')
df.head(5)

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101500 entries, 0 to 101499
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         101500 non-null  int64  
 1   user_id            101500 non-null  int64  
 2   hour               101500 non-null  int64  
 3   os                 101500 non-null  object 
 4   order_class        101500 non-null  object 
 5   surge              91431 non-null   object 
 6   app_opened         101500 non-null  int64  
 7   price_seen         101500 non-null  int64  
 8   order_made         101500 non-null  int64  
 9   ride_completed     101500 non-null  int64  
 10  user_cancelled     101500 non-null  int64  
 11  city_center_order  101500 non-null  int64  
 12  distance           91431 non-null   float64
 13  age                101500 non-null  int64  
 14  rfm                101500 non-null  object 
dtypes: float64(1), int64(10), object(4)
memory usage: 1

2) Выбираем генеральную совокупность пользователей (=Android)

In [ ]:
# создаю новый датафрейм, где преставлены только Android юзеры
df_new = df[df['os'].isin(['Android'])]
df_new.head(5)

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low
5,5,139455,5,Android,comfort,NaN,1,0,0,0,0,1,NaN,24,high


In [ ]:
# группирую датафрейм поюзерно, и считаю сумму по заказам и отменам
df_new = df_new.groupby(['user_id']).agg({'order_made': "sum", 'user_cancelled': "sum"})
df_new.head(10)

,order_made,user_cancelled
user_id,,
100093,2,0
100118,2,1
100134,3,1
100166,4,1
100255,1,1
100275,1,0
100303,1,0
100313,2,1
100375,3,2


3) Ключевая метрика - конверсия в отказы. По результату тестов если конверсия снизится, значит, наша гипотеза оказалась верна

In [ ]:
# Добавляю новый столбец в датафрейм - конверсию в отказы (user_cancelled / order_made)
df_new['cancel_conv'] = df_new['user_cancelled']/df_new['order_made'] * 100
df_new.head(10)

,order_made,user_cancelled,cancel_conv
user_id,,,
100093,2,0,0.000000
100118,2,1,50.000000
100134,3,1,33.333333
100166,4,1,25.000000
100255,1,1,100.000000
100275,1,0,0.000000
100303,1,0,0.000000
100313,2,1,50.000000
100375,3,2,66.666667


In [ ]:
# оставляю только тех пользователей, которые делали отмены (убираю нули)
df_new = df_new[df_new['cancel_conv'] > 0]
df_new.head(5)

,order_made,user_cancelled,cancel_conv
user_id,,,
100118,2,1,50.000000
100134,3,1,33.333333
100166,4,1,25.000000
100255,1,1,100.000000
100313,2,1,50.000000


In [ ]:
df_new.describe()

,order_made,user_cancelled,cancel_conv
count,6345.000000,6345.000000,6345.000000
mean,2.449961,1.158865,58.034973
std,1.217022,0.409122,28.109487
min,1.000000,1.000000,10.000000
25%,2.000000,1.000000,33.333333
50%,2.000000,1.000000,50.000000
75%,3.000000,1.000000,100.000000
max,10.000000,4.000000,100.000000


In [ ]:
# в обновленном датасете осталось 6345 строк
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6345 entries, 100118 to 999878
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_made      6345 non-null   int64  
 1   user_cancelled  6345 non-null   int64  
 2   cancel_conv     6345 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 198.3 KB


4) Формирование выборок - разбивка юзеров на две группы с помощью рандома

In [ ]:
import numpy as np

In [ ]:
# добавляю новую колонку, где каждому юзеру присвается рандомное значение от 0 до 1
df_new['rand'] = np.random.random(size=(6345))
df_new.describe()

,order_made,user_cancelled,cancel_conv,rand
count,6345.000000,6345.000000,6345.000000,6345.000000
mean,2.449961,1.158865,58.034973,0.500839
std,1.217022,0.409122,28.109487,0.286485
min,1.000000,1.000000,10.000000,0.000083
25%,2.000000,1.000000,33.333333,0.255307
50%,2.000000,1.000000,50.000000,0.504648
75%,3.000000,1.000000,100.000000,0.747488
max,10.000000,4.000000,100.000000,0.999681


In [ ]:
# формирую 2 выборки на основе столбца rand 
df_1 = df_new[df_new['rand'] < 0.5]
df_2 = df_new[df_new['rand'] > 0.5]


In [ ]:
print('Выборкки имеют одинаковую длину: ', len(df_1) == len(df_2))

Выборкки имеют одинаковую длину:  False


5) Проверка на однородность

In [ ]:
from scipy.stats import mannwhitneyu

alpha = 0.05         #уровень значимости                  

In [ ]:
# проверяю на однородность с помощью рангового критерия Манна-Уитни
result = mannwhitneyu(df_1['cancel_conv'], df_2['cancel_conv'])
print('p-value: %.8f' % result.pvalue)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

p-value: 0.34381822
Не отвергаем нулевую гипотезу
